# Laboratorio 4

## Equipo U

#### Ana Laura Correa Cabrales
#### Carlos Eduardo Ochoa Arambula
#### Pablo Francisco Vargas Castellanos
#### Emilio De La Torre Jimenez

El propósito: Que puedas describir los efectos cuantitativos en el precio durante los comunicados de indicadores macroeconómicos, así como el detectar oportunidades de inversión a partir de los resultados obtenidos. Lo anterior, utilizando lo que se conoce como "Calendario Económico", y más en particular, aquel que incorpora información de las principales economías del mundo por lo que es ampliamiente utilizado en el mercado de Forex.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
from collections import Counter
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [36]:
#Cargamos los datos, y eliminamos los que no tienen valor
df = pd.read_csv("calendario_economico.csv") 
df.dropna(inplace = True)


In [3]:
#buscamos y agrupamos nuestros datos en Data Frames
data1=df.loc[df['Name'] == 'Industrial Production s.a. (MoM)']
IndProduction=data1.loc[df['Country'] == 'European Monetary Union']
IndProduction=IndProduction.reset_index(drop=True)
data2=df.loc[df['Name'] == 'Retail Sales (MoM)']
RetailSales=data2.loc[df['Country'] == 'European Monetary Union']
RetailSales=RetailSales.reset_index(drop=True)


In [4]:
#Date time a la variable "timestamp" para poder despues buscarla en los precios 
IndProduction['timestamp']=pd.to_datetime(IndProduction['timestamp'])
IndProduction['timestamp']=IndProduction['timestamp'].dt.tz_localize('UTC')
RetailSales['timestamp']=pd.to_datetime(RetailSales['timestamp'])
RetailSales['timestamp']=RetailSales['timestamp'].dt.tz_localize('UTC')

In [5]:
#definimos escenarios por restricciones
def escenario(x):
    x['escenario']= x['actual']
    for i in range(len(x)):
        if x['actual'][i] >= x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i] = "A"
        elif x['actual'][i] >= x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="B"
        elif x['actual'][i] < x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i]="C"
        elif x['actual'][i] < x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="D"
    return x
    

In [65]:
RS=escenario(RetailSales)
IP=escenario(IndProduction)
RS

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario
0,2009-01-09 10:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,0.32,0.30,-0.1,B
1,2009-02-04 10:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,-0.90,0.32,-0.2,D
2,2009-03-13 10:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,0.10,-0.90,0.2,C
3,2009-04-06 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,-1.00,0.10,-0.3,D
4,2009-05-06 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,-0.30,-1.00,0.1,C
...,...,...,...,...,...,...,...,...,...
124,2019-05-06 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,0.00,0.40,-0.1,B
125,2019-06-05 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,-0.40,0.00,-0.4,B
126,2019-07-04 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,-0.30,-0.40,0.3,C
127,2019-08-02 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,1.10,-0.30,0.2,A


In [38]:
#cargamos los datos de los precios del Euro-Dolar
df = pd.read_csv("precios_historicos_eurusd.csv") 
df['timestamp']=pd.to_datetime(df['timestamp'])
df['timestamp']=df['timestamp'].dt.tz_localize('UTC')


In [60]:
#checando valores en precios EUR/USD con misma hora que nuestro calendario economico
#Data frame con los valores que coinciden las horas
comparacionRS=df[df.timestamp.isin(RS.timestamp)]
comparacionIP=df[df.timestamp.isin(IP.timestamp)]

In [77]:
periodos = [df.iloc[i-6:i+7]for i in comparacionRS.index]

o (Dirección) Close (t_30) - Open(t_0)

o (Pips Alcistas) High(t_0 : t_30) – Open(t_0)

o (Pips Bajistas) Open(t_0) – Low(t_0 : t_30)

o (Volatilidad) High(t_-30 : t_30) , - mínimo low (t_-30:t_30)

In [121]:
direcc = periodos[0]["close"].iloc[12] - periodos[0]["open"].iloc[6]
pipsalcis = periodos[0]["high"].iloc[6:13].max() - periodos[0]["open"].iloc[0]
pipsbaj = periodos[0]["open"].iloc[0] - periodos[0]["low"].iloc[6:13].min()
vol = periodos[0]["high"].max()-periodos[0]["low"].min()
